In [17]:
import pandas as pd
from plotly import graph_objects as go
import re

In [18]:
df_salaries = pd.read_csv('C:\\Users\\iKerng\\Python\\School_DA\\final_task\\data\\Salaries.csv', delimiter=',')
df_salaries = df_salaries[['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay',
                           'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Status']]
df_work = df_salaries.copy()

In [19]:
def str_nice(name=''):
    if type(name) == str:
        name = name.lower()
        name = re.sub(r'\s{2,}', ' ', name)
        name = re.sub(r'[^\w ]|\d', '', name)
    return name

In [20]:
df_work.EmployeeName = df_work.EmployeeName.apply(str_nice)
df_work.JobTitle = df_work.JobTitle.apply(str_nice)
df_work = df_work[df_work.EmployeeName != 'not provided']
df_work.BasePay = df_work.BasePay.astype(float)
df_work.OvertimePay = df_work.OvertimePay.astype(float)
df_work.OtherPay = df_work.OtherPay.astype(float)
df_work.Benefits = df_work.Benefits.astype(float)
df_work.TotalPay = df_work.TotalPay.astype(float)
df_work.TotalPayBenefits = df_work.TotalPayBenefits.astype(float)

In [13]:
ls_job_titles = df_work[~(df_work.JobTitle.str.contains('fire')
        | df_work.JobTitle.str.contains('police')
        | df_work.JobTitle.str.contains('medicine'))].JobTitle.apply(lambda x: re.split(' ', x)).to_list()
job_title_words = []
for i in ls_job_titles:
    for k in i:
        job_title_words.append(k)
(pd.DataFrame(job_title_words, columns=['job_words']).
 groupby(by='job_words')[['job_words']].count()).rename(columns={'job_words':'quantity'}).\
    reset_index().job_words.to_list()
#
# ls_clean_words = ['acupuncturist', 'anesthetist', 'captain', 'clincal', 'clinical', 'commissioner', 'criminalist',
#  'dental', 'dentist', 'diagnostic', 'dietetic', 'dietitian', 'disease', 'emerg', 'emergency', 'emergencycy',
#  'epidemiologist', 'forensic', 'hlth', 'hospital', 'hygienist', 'injury', 'inpsector', 'inspection',
#  'inspections', 'inspector', 'fire', 'police', 'medicine']
ls_clean_words = ['acupuncturist', 'anesthetist', 'clincal', 'clinical', 'dental', 'dentist', 'diagnostic', 'dietetic'
    , 'dietitian', 'disease', 'emerg', 'emergency', 'emergencycy', 'epidemiologist', 'forensic', 'hlth', 'hospital'
    , 'hygienist', 'injury', 'sheriff', 'captain', 'criminalist', 'fire', 'police', 'medic']
str_contains_words = str(ls_clean_words).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')
# df_work_dep = df_work[df_work.JobTitle.str.contains(str_contains_words)]

In [14]:
ls_clean_words_med = ['acupuncturist', 'anesthetist', 'clincal', 'clinical', 'dental', 'dentist', 'diagnostic'
    , 'dietetic', 'dietitian', 'disease', 'emerg', 'emergency', 'emergencycy', 'epidemiologist', 'forensic', 'hlth'
    , 'hospital', 'hygienist', 'medic']
ls_clean_words_fire = ['fire']
ls_clean_words_police = ['sheriff', 'criminalist', 'police']

str_word_med = str(ls_clean_words_med).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')
str_word_fire = str(ls_clean_words_fire).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')
str_word_police = str(ls_clean_words_police).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')

ls_fire = df_work[df_work.JobTitle.str.contains(str_word_fire)].index.to_list()
ls_med = df_work[df_work.JobTitle.str.contains(str_word_med)
        & ~(df_work.JobTitle.str.contains(str_word_fire)
            | df_work.JobTitle.str.contains(str_word_police))].index.to_list()
ls_police = df_work[df_work.JobTitle.str.contains(str_word_police)
        & ~(df_work.JobTitle.str.contains(str_word_med) | df_work.JobTitle.str.contains(str_word_fire))].index.to_list()

df_work['department'] = None
df_work.department.iloc[ls_fire] = 'fire'
df_work.department.iloc[ls_med] = 'medic'
df_work.department.iloc[ls_police] = 'police'

C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
df_work_dep = df_work[df_work.department.notna()].copy()
df_work_dep = df_work_dep[['department', 'Year', 'BasePay']]
df_work_graf = df_work_dep.groupby(by=['department', 'Year']).aggregate([max, 'median', 'mean'])['BasePay'].reset_index()
df_work_graf = df_work_graf.rename(columns={'max':'Maximum', 'median':'Median', 'mean':'Average'})

### Изменение оклада сотрудников различных департаментов (максимальная, медианная и средняя).

In [16]:
for i in ['Maximum','Median','Average']:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_work_graf[df_work_graf.department == 'police']['Year'],
                             y=df_work_graf[df_work_graf.department == 'police'][i], name='Police Department'))
    fig.add_trace(go.Scatter(x=df_work_graf[df_work_graf.department == 'fire']['Year'],
                             y=df_work_graf[df_work_graf.department == 'fire'][i], name='Fire Department'))
    fig.add_trace(go.Scatter(x=df_work_graf[df_work_graf.department == 'medic']['Year'],
                             y=df_work_graf[df_work_graf.department == 'medic'][i], name='Medical Department'))
    fig.update_layout(title=i + ' BasePay by year',
                      xaxis_title='Year',
                      yaxis_title='Basepay $')
    fig.show()